# Autocorrect (Viterbi)
#### Authors:
v1.0 (2017 Spring) Tavor Baharav, Kabir Chandrasekhar, Sinho Chewi, Andrew Liu, Kamil Nar, David Wang, and Kannan Ramchandran

v1.1 (2017 Fall) Sinho Chewi, Avishek Ghosh, Chen Meng, Abhay Parekh, and Jean Walrand

v1.2 (2018 Spring) Tavor Baharav, Kaylee Burns, Gary Cheng, Sinho Chewi, Hemang Jangle, William Gan, Alvin Kao, Chen Meng, Vrettos Muolos, Kanaad Parvate, Ray Ramamurti

## Hidden Markov Model

In this lab you will be implementing autocorrect using the Viterbi algorithm.

We will model the English language as a HMM. The state space is the set of all English words. The *hidden states* are the intended words of the writer, while the *emissions* are the actual words written down, which include typos. As an example, suppose an author intends to write the sentence "the dog barks". The sequence of hidden states is

$$X(0) = \text{the},$$
$$X(1) = \text{dog},$$
$$X(2) = \text{barks}.$$

However, the author is prone to making errors while typing, so the emissions (sometimes called observations) are

$$Y(0) = \text{the},$$
$$Y(1) = \text{dog},$$
$$Y(2) = \text{barjs}.$$

The Viterbi algorithm gives the most likely sequence of hidden states, i.e. the most likely sequence of intended words which produced the typo-filled output text.

## Transition Model
The transition model gives the probability of the transition from the hidden state $x(i)$ to the hidden state $x(i+1)$. 

In the field of natural language processing (NLP), the literature often mentions *$n$-grams* models. A *unigram* is a single word. A *bigram* is a pair of words which appear consecutively in a text. You will use the unigram counts to determine the relative frequencies of words. The word frequencies specify the initial distribution of the HMM. The bigram counts are used to infer word-to-word transition probabilities.

We will be using data from the source http://norvig.com/ngrams/ for the unigram and bigram counts. We took the unigrams from the file "count_1w100k.txt" and the bigrams from the file "count_2w.txt".

You will be building a list of initial distribution of 10000 words, and the word-to-word transition matrix.

In [1]:
# Importing everything here
import math
import numpy as np
import time

In [ ]:
# In order to speed up the process, we limit the number of words to a small subset of the dataset, based on the most frequently appearing words.
# We chose to limit the number of words to 10,000, but you can play around with this number too.
NUM_WORDS = 10000

# Build the initial distribution of the HMM
# i.e. the count of each word divided by the sum of counts of all words
pi_0 = np.zeros(NUM_WORDS)
# key: word, value: an integer ID of the word
word_to_id = {}
# a list of all the words
word_list = []



with open("count_1w100k.txt") as f:
    for i in range(NUM_WORDS):
        line = f.readline()
        tokens = line.lower().split()
        word = tokens[0]
        count = float(tokens[1])
        
        #Your code here
        # Construct pi_0, word_to_id and word_list


pi_0 = pi_0 / np.sum(pi_0)
log_pi_0 = np.log(pi_0)

In [ ]:
# Build the word-to-word transition matrix
# this process is similar to what you did in MCMC lab (except you used "War and Peace")
P = np.zeros((NUM_WORDS, NUM_WORDS))


with open("count_2w.txt") as f:
    for line in f:
        tokens = line.lower().split()
        word1 = tokens[0]
        word2 = tokens[1]
        count = float(tokens[2])
        if word1 in word_to_id and word2 in word_to_id:
            #Your code here
            # Construct P


# log_P is your transition distance matrix
P = P / np.sum(P)
log_P = np.log(P)

In [ ]:
# Sanity Checks
# probability of seeing 'probability' is much smaller than probability of seeing 'the'
print("Probability of 'the'", pi_0[word_to_id["the"]])
print("Probability of 'probability'", pi_0[word_to_id["probability"]])


# probability of 'you' followed by 'is' should be much smaller than Probability of 'you' followed by 'are'
print("Probability of 'you' followed by 'are'", P[word_to_id["you"], word_to_id["are"]])
print("Probability of 'you' followed by 'is'", P[word_to_id["you"], word_to_id["is"]])

## Emission Model: Edit Distance

The emission model gives the probability of seeing a word $y(i)$ when the true hidden state is $x(i)$. The emission probabilities are based on a measure of string dissimilarity known as *edit distance* or *Levenshtein distance*.

$\underline{Inputs}$ Two strings s_1[1...n], s_2[1...m]

$\underline{Goal}$ Number of characterwise Inserts, Deletes and Substitutes required to edit s_1[1...n] into s_2[1...m]

$\underline{Definition}$
The edit distance between two strings $s_1$ and $s_2$ (with lengths $m$ and $n$ respectively) is defined by the recursive equations

$$d(s_1, s_2) =
\begin{cases}
  m & \text{if } s_2 \text{ is empty}, \\
  n & \text{if } s_1 \text{ is empty}, \\
  \min
  \begin{cases}
    d(s_1(2, \dotsc, m), s_2) + 1 \\
    d(s_1, s_2(2, \dotsc, n)) + 1 \\
    d(s_1(2, \dotsc, m), s_2(2, \dotsc, n)) + \mathbf{1}\{s_1(1) \ne s_2(1)\}
  \end{cases}
  & \text{otherwise}.
\end{cases}
$$
The notation is as follows: $s_1(1)$ represents the first character in the first string; and $s_1(2, \dotsc, n)$ represents the substring which starts at the second character and ends at the $n$th character of $s_1$. Similarly for $s_2$.

Here is some $\underline{intuition}$ for edit distance: it represents the minimum number of operations to change $s_1$ into $s_2$, where an operation is either
- adding a new character,
- deleting an existing character,
- or changing one character into another.

You can simply think of edit distance as a way to measure the distance between two strings. An example is shown below, demonstrating that the edit distance between "elephant" and "relevant" is 3, which is achieved by taking "relevant", deleting the r, replacing the v with a p, and adding an h immediately after. Convince yourself that the following matrix makes sense. Fill it out top to bottom, left to right.
<img src = "https://hsto.org/storage2/74b/c0f/a85/74bc0fa858652701ff47bfd125c83eeb.png">

[This link](https://nlp.stanford.edu/IR-book/html/htmledition/edit-distance-1.html) may be a helpful resource for understanding implementing the edit distance. It also includes a pseudo code example.

In [ ]:
# A function that takes in two words and return the edit distance between them.
# Helper function used to calculate the emission probability
def edit_distance(word1, word2):
    m = len(word1)
    n = len(word2)
    d = np.zeros((m + 1, n + 1))
    for i in range(m + 1):
        # Your code here
    for j in range(1, n + 1):
        # Your code here
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            # Your code here
    return d[m, n]

You just implemented the idea of dynamic programming in edit_distance function! Dynamic programming is an algorithm that uses a table to store intermediate values as it builds up the probability of the observation sequence. A concrete example of a dynamic programming problem is to find the longest path in a DAG (directed acyclic graph). Viterbi algorithm is itself another example of dynamic programming. You start from the end of the sequence and find the node that gives the shortest distance at each level moving backward. At each level, you assume you have the info about the shortest distance from each node at the next level to the end of the sequence, and you use that to calculate the shortest distance for the current level. This is exactly dynamic programming!

The emission model is given by a Poisson distribution based on the edit distance. If the hidden state $x$ and the emission $y$ have edit distance $d(x, y) = k$, then the emission probability is

$$Q(x, y) = \frac{e^{-\lambda} \lambda^k}{k!}.$$

The larger the edit distance, the smaller the emission probability, which aligns with our intuition. 

Implement a function that gives matrix log_Q = log(P), where P is the matrix of emission probability based on the emission model above. log_Q is of dimension (n, m), where n is the number of words in the sentence, and m is the number of potential words for decoding. You may want to play around with different values of $\lambda$ for the poisson distribution to see which one works best. We tried using $\lambda = 0.01$.

In [ ]:
# Try play around with this number
RATE = 0.01

# Words is a list of lower-case words in the sentence we are trying to decode/autocorrect
# n: the number of words in the sentence
# m: the number of potential words for decoding
# log_Q is your emission distance matrix
def emission_distance_matrix(words): 
    count = len(words)
    log_Q = np.zeros((count, NUM_WORDS))
    for i in range(count):
        word = words[i]
        for j in range(NUM_WORDS):
            dist = #Your code here
            log_Q[i, j] = #Your code here
    return log_Q


## Implementation of Viterbi

Now that you have a function for log emission distance matrix, a list of initial distribution of 10000 words, and the word-to-word transition matrix, it's finally time to implement the viterbi algorithm to do some autocorrect! Try to use NumPy's functions whenever possible because they are *vectorized* (optimized for high performance).

Good luck and have fun!

In [ ]:
def viterbi(sentence):
    words = sentence.lower().split()
    count = len(words)
    
    log_Q = emission_distance_matrix(words)

    # V[i,j] denotes the min distance of seeing word j at position i
    V = np.zeros((count, NUM_WORDS))
    # pointers[i,j] denotes which word to go to at position i+1 to achieve the min distance of seeing word j at position i
    pointers = np.zeros((count - 1, NUM_WORDS))

    
    for i in range(count - 2, -1, -1):
        for j in range(NUM_WORDS):
            #Your code here
            # Construct V and pointers

    start_state = np.argmin(V[0] - log_pi_0 - log_Q[0])
    sentence = [word_list[int(start_state)]]
    curr_state = start_state
    
    for i in range(count - 1):
        #Your code here
        # Construct sentence

    return " ".join(sentence)

In [ ]:
# Check the final result!
# Staff solution returns "why is the sky blue",
# takes under 4 seconds to run
start = time.time()
print(viterbi("whyy is th ski bluui"))
end = time.time()
print("Execution Time:", end - start)

Reference: https://www.cs.sfu.ca/~mori/courses/cmpt310/a5.pdf